In [9]:
import numpy as np
import cv2
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.optimizers import SGD, Adam
from keras.models import Sequential, Graph
from keras.layers import Convolution2D, ZeroPadding2D, MaxPooling2D
from keras.layers import normalization
import h5py
from test import *
from keras import backend as K
import time
from FaceRec.pretrained_cnn import *                         # Hi Shivam, this is going to take about a second
import FaceRec
from FaceRec.net import *
from FaceRec.get_input import *
from scipy.misc import imsave
import numpy as np


In [10]:
x,y = from_file()                             # This prints some things on the terminal
x_test, y_test = test_file()                  # last statement must be : No. of images 237 no. of classes: 237

In [11]:
model = Sequential()
model.add(Convolution2D(64, 3, 3, input_shape=(3, 227, 227),
                        activation='relu', name='conv1_1', border_mode='same'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(64, 3, 3, activation='relu', name='conv1_2'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(128, 3, 3, activation='relu', name='conv2_1'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(128, 3, 3, activation='relu', name='conv2_2'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_1'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_2'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_3'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_1'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_2'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_3'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_1'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_2'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_3'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))


graph = Graph()
graph.add_input(name='input1', input_shape=(3, 227, 227))
graph.add_node(model, name='cnn', input='input1')

layer_dict = dict([(layer.name, layer) for layer in graph.nodes['cnn'].layers])
# print layer_dict

In [12]:
PRETRAINED = "cnn_weights.h5"
print 'Loading weights ...'
start=time.time()
f = h5py.File(PRETRAINED)
for k in range(f.attrs['nb_layers']):
    if k >= len(graph.nodes['cnn'].layers):
        # we don't look at the last (fully-connected) layers in the savefile
        break
    g = f['layer_{}'.format(k)]
    weights = [g['param_{}'.format(p)] for p in range(g.attrs['nb_params'])]
    graph.nodes['cnn'].layers[k].set_weights(weights)
f.close()
print 'Model loaded in ..',time.time()-start

In [14]:
graph.add_node(Flatten(), input='cnn', name='flatten1')
graph.add_node(Dense(output_dim=4096, activation='relu'), name='dense1', input='flatten1')
graph.add_node(Dense(output_dim=4096, activation='relu'), name='dense2', input='dense1')
graph.add_node(Dropout(0.5), name='dropout1', input='dense2')
graph.add_node(Dense(output_dim=21, activation='softmax'), name='denseOut', input='dropout1')
graph.add_output(name='output', input='denseOut')

In [16]:
print 'Normalizing data...'
x = x.astype('float32')
x_test = x_test.astype('float32')
x /= 255
x_test /= 255
y = np_utils.to_categorical(y, 21)
y_test = np_utils.to_categorical(y_test, 21)

IndexError: index 7 is out of bounds for axis 1 with size 7